<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hashing-PII-columns-in-data" data-toc-modified-id="Hashing-PII-columns-in-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hashing PII columns in data</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Setup---Imports" data-toc-modified-id="Setup---Imports-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup - Imports</a></span></li><li><span><a href="#Setup---Files-and-Directories" data-toc-modified-id="Setup---Files-and-Directories-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setup - Files and Directories</a></span></li><li><span><a href="#Setup---Initialize-HMACHasher-with-salts" data-toc-modified-id="Setup---Initialize-HMACHasher-with-salts-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Setup - Initialize HMACHasher with salts</a></span><ul class="toc-item"><li><span><a href="#shared-HMACHasher" data-toc-modified-id="shared-HMACHasher-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>shared HMACHasher</a></span></li></ul></li><li><span><a href="#Setup---Functions" data-toc-modified-id="Setup---Functions-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Setup - Functions</a></span><ul class="toc-item"><li><span><a href="#Functions---hash-functions" data-toc-modified-id="Functions---hash-functions-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>Functions - hash functions</a></span></li></ul></li></ul></li><li><span><a href="#Hash-data" data-toc-modified-id="Hash-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Hash data</a></span><ul class="toc-item"><li><span><a href="#function-parse_fixed_width_record" data-toc-modified-id="function-parse_fixed_width_record-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>function parse_fixed_width_record</a></span></li><li><span><a href="#fixed-width-field-spec" data-toc-modified-id="fixed-width-field-spec-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>fixed-width field spec</a></span></li><li><span><a href="#hashing-spec" data-toc-modified-id="hashing-spec-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>hashing spec</a></span></li><li><span><a href="#hash!" data-toc-modified-id="hash!-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>hash!</a></span></li></ul></li><li><span><a href="#Evaluate" data-toc-modified-id="Evaluate-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Evaluate</a></span></li></ul></div>

# Hashing PII columns in data

This example notebook hashes the Name fields and SSN in a sample data file.

In this example a set of data files are broken into multiple files, one file per quarter, one row per unit of interest.  This code reads directly from the original files row by row, hashing column values for each row and building an output row with same number of columns, but hashed values where desired, then writing each row to an output file.

Logic overiew - for each row in fixed-width file:

- reads row from original fixed-width CSV file into a row value list.
- makes a copy of the row value list, for output.
- pulls in the fields to hash, hashes them, then replaces existing values in output list with hashed values.
- writes hashed row to output CSV.

# Setup

- Back to [Table of Contents](#Table-of-Contents)

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
import copy
import csv
import datetime
import glob
import hashlib
import six
import uuid

print( "Imports imported at " + str( datetime.datetime.now() ) )

## Setup - Files and Directories

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
pwd

In [ ]:
# work directories
#root_directory = "workspace"
#configuration_directory = "configuration"
#work_directory = root_directory + "work"
#data_directory = root_directory + "ingest"
#source_directory = data_directory + "original_data"
#output_directory = root_directory + "hashed_output"

# can get fancy, for example, all are current directory.
root_directory = "."
configuration_directory = root_directory + "/examples"
work_directory = "." # needs to be a directory that has the hmac_hasher folder that sits alongside this file in the repository inside of it.
data_directory = root_directory + "/examples"
source_directory = data_directory
output_directory = "."

# variable names used in the code below.
input_file_directory_path = source_directory
output_file_directory_path = output_directory

print( "Directories configured at " + str( datetime.datetime.now() ) )

## Setup - Initialize HMACHasher with salts

- Back to [Table of Contents](#Table-of-Contents)

We have a shared HMAC passphrase we will use for hashing.  We can just set the object up here, then use map to call it on each column we need to hash.

In [ ]:
# first, load the HMACHasher class.
hmac_hasher_folder_path = work_directory + "/hmac_hasher"
hmac_hasher_class_file_path = hmac_hasher_folder_path + "/hmac_hasher.py"

Use the "%run" command to run the Python file that defines the HMACHasher class and load the class into memory.

In [ ]:
%run $hmac_hasher_class_file_path

print( "HMACHasher class imported from {} at {}".format( hmac_hasher_class_file_path, str( datetime.datetime.now() ) ) )

### shared HMACHasher

Sometimes you will have a secret per field or field type.  In this case, we are using a single secret for all fields, so only need one configuration and one hasher.

The secret is stored in a configuration INI file, so it can persist but not be explicitly present in code files.

The minimal configuration needed for this is a "[secret]" section that contains a "passphrase".  An example:

    [secret]

    passphrase=fakedata

    [file_paths]

    input_file_path=./test_data.txt
    ;output_file_path=./hashed_output.csv

    [configuration]
    has_header_row=true

The "[file_paths]" and "[configuration]" are for using this program in standalone mode.  We don't need those here, and we are responsible for dealing with IO and file reading.

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
# make single instance of the HMACHasher for all values.
my_hasher = HMACHasher()

# load the passphrase/salt from the configuration file
hmac_hasher_ini_file_path = configuration_directory + "/hashing_configuration.ini"

# store configuration file path in HMACHasher, then load config.
my_hasher.configuration_ini_file_path = hmac_hasher_ini_file_path
config_load_messages = my_hasher.load_configuration_from_ini_file()

# errors?
if ( len( config_load_messages ) > 0 ):

    # errors.
    for error_message in config_load_messages:
        
        print( "- " + str( error_message ) )
        
    #-- END loop over errors. --#

else:
    
    print( "Config loaded from path " + str( hmac_hasher_ini_file_path ) + " at " + str( datetime.datetime.now() ) )
    
#-- END check for errors loading configuration. --#

print( "HMACHasher instance created at " + str( datetime.datetime.now() ) )

In [ ]:
# test hashing a value.
expected_value = ""

# On first run, this will not match.  To test:
# - Copy value from first run into expected_value, above.
# - stop kernel and clear output.
# - run all cells again.

test_value = "Exculpatory"
test_hash = ""
test_hash = my_hasher.hash_value( test_value )
print( "FROM " + str( test_value ) + " TO " + str( test_hash ) )
print( "Equal to expected?: " + str( expected_value == test_hash ) )

## Setup - Functions

- Back to [Table of Contents](#Table-of-Contents)

### Functions - hash functions

- Back to [Table of Contents](#Table-of-Contents)

Here, we set up a function per type of value.  If you had multiple secrets used within a given file, each function could default to a different hasher.  Here, we default all to the same since we are using a single secret.

In [ ]:
def hash_ssn( value_IN, hasher_IN = my_hasher ):
    
    # return reference
    hash_OUT = ""
    
    # hash using SSN method:
    # - removes punctuation
    # - replaces multiple spaces with a single space
    # - strips white space from ends
    hash_OUT = hasher_IN.hash_ssn_value( value_IN )
    
    return hash_OUT

#-- END function hash_ssn() --#

print( "Function hash_ssn() declared at " + str( datetime.datetime.now() ) )

    
def hash_name( value_IN, hasher_IN = my_hasher ):
    
    # return reference
    hash_OUT = "" 
     
    # hash using name method to standardize:
    # - converts to upper case
    # - removes punctuation
    # - replaces multiple spaces with a single space
    # - strips white space from ends
    hash_OUT = hasher_IN.hash_name_value( value_IN )
    
    return hash_OUT

#-- END function hash_name() --#

print( "Function hash_name() declared at " + str( datetime.datetime.now() ) )

# Hash data

- Back to [Table of Contents](#Table-of-Contents)

## function parse_fixed_width_record

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
def parse_fixed_width_record( record_string_IN, fixed_width_spec_list_IN ):
    
    '''
    Accepts fixed-width file line string and list of field spec dictionaries of fields to be extracted.
    For each, in the order they appear in the list, extracts field from record string and
        appends it to output list.  Returns list of values.
    '''
    
    # CONSTANTS-ish
    FIXED_WIDTH_SPEC_NAME = "name"
    FIXED_WIDTH_SPEC_INDEX = "index"
    FIXED_WIDTH_SPEC_START = "start"
    FIXED_WIDTH_SPEC_END = "end"

    # return reference
    value_list_OUT = None
    
    # declare variables
    record_string = ""
    field_counter = -1
    fixed_width_spec_list = []
    fixed_width_spec = {}
    field_name = None
    field_index = None
    field_start = None
    field_end = None
    start_index = -1
    end_index = -1
    field_value = None

    # Make sure there is a value
    if ( ( record_string_IN is not None ) and ( record_string_IN != "" ) ):
        
        # record string
        record_string = record_string_IN
        
        print( "\n====> record string: {}".format( record_string ) )
        
        # init
        value_list_OUT = []

        # loop over spec
        field_counter = 0
        fixed_width_spec_list = fixed_width_spec_list_IN
        for fixed_width_spec in fixed_width_spec_list:
            
            # get values
            field_name = fixed_width_spec[ FIXED_WIDTH_SPEC_NAME ]
            field_index = fixed_width_spec[ FIXED_WIDTH_SPEC_INDEX ]
            field_start = fixed_width_spec[ FIXED_WIDTH_SPEC_START ]
            field_end = fixed_width_spec[ FIXED_WIDTH_SPEC_END ]
            
            # convert start and end to 0-indexed slice indices:
            start_index = field_start - 1
            end_index = field_end
            
            # get field value
            field_value = record_string[ start_index : end_index ]
            
            # strip white space
            field_value = field_value.strip()
            
            # append to output list.
            value_list_OUT.append( field_value )
            
            # sanity check - in desired index?
            print( "- field {} = {}; index = {}; desired index = {}".format( field_name, field_value, field_counter, field_index ) )
            
            # increment counter
            field_counter += 1
            
        #-- END loop over fields. --#

    #-- END check to see if string passed in. --#
    
    return value_list_OUT

#-- END function parse_fixed_width_record() --#

print( "Function parse_fixed_width_record() declared at " + str( datetime.datetime.now() ) )

## fixed-width field spec

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
# define fixed-width field spec list

# CONSTANTS-ish
FIXED_WIDTH_SPEC_NAME = "name"
FIXED_WIDTH_SPEC_INDEX = "index"
FIXED_WIDTH_SPEC_START = "start"
FIXED_WIDTH_SPEC_END = "end"

# declare variables
field_name = ""
fixed_width_field_list = []
fixed_width_spec = {}

# ==> init fixed-width spec.
# - Must include all columns we care to extract, in the order we want them to appear in list.

# ID, index 0 - 1-9 (so 0-8)
field_name = "ID"
fixed_width_spec = {}
fixed_width_spec[ FIXED_WIDTH_SPEC_NAME ] = field_name
fixed_width_spec[ FIXED_WIDTH_SPEC_INDEX ] = 0
fixed_width_spec[ FIXED_WIDTH_SPEC_START ] = 1
fixed_width_spec[ FIXED_WIDTH_SPEC_END ] = 9
fixed_width_field_list.append( fixed_width_spec )

# first_name, index 1 - 10-24 (so 9-23)
field_name = "first_name"
fixed_width_spec = {}
fixed_width_spec[ FIXED_WIDTH_SPEC_NAME ] = field_name
fixed_width_spec[ FIXED_WIDTH_SPEC_INDEX ] = 1
fixed_width_spec[ FIXED_WIDTH_SPEC_START ] = 10
fixed_width_spec[ FIXED_WIDTH_SPEC_END ] = 24
fixed_width_field_list.append( fixed_width_spec )

# middle_name, index 2 - 25 (so 24)
field_name = "middle_name"
fixed_width_spec = {}
fixed_width_spec[ FIXED_WIDTH_SPEC_NAME ] = field_name
fixed_width_spec[ FIXED_WIDTH_SPEC_INDEX ] = 2
fixed_width_spec[ FIXED_WIDTH_SPEC_START ] = 25
fixed_width_spec[ FIXED_WIDTH_SPEC_END ] = 25
fixed_width_field_list.append( fixed_width_spec )

# last_name, index 3 - 26-45 (so 25-44)
field_name = "last_name"
fixed_width_spec = {}
fixed_width_spec[ FIXED_WIDTH_SPEC_NAME ] = field_name
fixed_width_spec[ FIXED_WIDTH_SPEC_INDEX ] = 3
fixed_width_spec[ FIXED_WIDTH_SPEC_START ] = 26
fixed_width_spec[ FIXED_WIDTH_SPEC_END ] = 45
fixed_width_field_list.append( fixed_width_spec )



## hashing spec

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
# define simple hash specification - for each column to be hashed:
# - index
# - hash function to use
# - hasher to use

# CONSTANTS-ish
HASH_SPEC_INDEX = "index"
HASH_SPEC_HASH_FUNCTION = "hash_function"
HASH_SPEC_HASHER = "hasher"

# declare variables
hash_spec_list = []
hash_spec = {}

# ID, index 0 - 1-9 (so 0-8)
hash_spec = {}
hash_spec[ HASH_SPEC_INDEX ] = 0
hash_spec[ HASH_SPEC_HASH_FUNCTION ] = hash_ssn
hash_spec[ HASH_SPEC_HASHER ] = my_hasher
hash_spec_list.append( hash_spec )

# first_name, index 1 - 10-24 (so 9-23)
hash_spec = {}
hash_spec[ HASH_SPEC_INDEX ] = 1
hash_spec[ HASH_SPEC_HASH_FUNCTION ] = hash_name
hash_spec[ HASH_SPEC_HASHER ] = my_hasher
hash_spec_list.append( hash_spec )

# middle_name, index 2 - 25 (so 24)
hash_spec = {}
hash_spec[ HASH_SPEC_INDEX ] = 2
hash_spec[ HASH_SPEC_HASH_FUNCTION ] = hash_name
hash_spec[ HASH_SPEC_HASHER ] = my_hasher
hash_spec_list.append( hash_spec )

# last_name, index 3 - 26-45 (so 25-44)
hash_spec = {}
hash_spec[ HASH_SPEC_INDEX ] = 3
hash_spec[ HASH_SPEC_HASH_FUNCTION ] = hash_name
hash_spec[ HASH_SPEC_HASHER ] = my_hasher
hash_spec_list.append( hash_spec )


## hash!

- Back to [Table of Contents](#Table-of-Contents)

In [ ]:
# ==> example data

# declare variables - loop over files
file_list = []
file_path = ""
path_part_list = []
file_name = ""
temp_file_name = ""
file_year = ""
file_quarter = ""

# declare variables - process each file
path_separator = "/"
input_file = ""
input_file_match = ""
input_file_search_string = "*.txt"
input_file_encoding = "utf-8"
has_header_row = False
output_file = ""
line_counter = -1
hash_output_file = None
to_hash_csv_file = None
input_csv_reader = None
current_line = None
output_csv_writer = None
current_record = None

# values from record
current_value = ""
hashed_value = ""
row_value_list = []
current_hash_spec = None
hash_spec_index = -1
hash_spec_hash_function = None
hash_spec_hasher = None

# first get list of *.csv files in directory.
print( "Looking for files in {}".format( input_file_directory_path ) )
input_file_match = "{}/{}".format( input_file_directory_path, input_file_search_string )
file_list = glob.glob( input_file_match )
print( "File list: " + str( file_list ) )

for file_path in file_list:
    
    # Parse out the file name.
    path_part_list = file_path.split( path_separator )
    file_name = path_part_list[ -1 ]
    
    print( "--> Current file: {} @ {}".format( str( file_name ), str( datetime.datetime.now() ) ) )

    # initialize
    line_counter = 0
    input_file = file_path
    output_file = output_file_directory_path + "/hashed-" + file_name

    # open the output file for writing.
    with open( output_file, "w" ) as hash_output_file:

        # init CSV writer.
        output_csv_writer = csv.writer( hash_output_file, delimiter = "," )

        # open the input file for reading
        with open( input_file, encoding = input_file_encoding ) as to_hash_file:

            # output header row?
            if ( has_header_row == True ):
                
                # yes - output first row as is.
                current_record = to_hash_file.readline()
                row_value_list = parse_fixed_width_record( current_record, fixed_width_field_list )
                output_csv_writer.writerow( row_value_list )
                
            #-- END check to see if header row --#

            # loop over records
            for current_record in to_hash_file:
                
                # initialize values
                current_value = ""
                hashed_value = ""

                # initialize output list from input record
                row_value_list = parse_fixed_width_record( current_record, fixed_width_field_list )

                # increment line counter
                line_counter += 1

                # loop over hash spec
                for current_hash_spec in hash_spec_list:
                    
                    # read hash spec information
                    hash_spec_index = current_hash_spec[ HASH_SPEC_INDEX ]
                    hash_spec_hash_function = current_hash_spec[ HASH_SPEC_HASH_FUNCTION ]
                    hash_spec_hasher = current_hash_spec[ HASH_SPEC_HASHER ]

                    # get value for requested index
                    current_value = row_value_list[ hash_spec_index ]
                    
                    # hash it.
                    hashed_value = hash_spec_hash_function( current_value, hash_spec_hasher )
                    
                    # place hashed value in row value list.
                    row_value_list[ hash_spec_index ] = hashed_value
                    
                #-- END loop over hash spec list. --#
                    
                # write to output file.
                output_csv_writer.writerow( row_value_list )

                if ( ( line_counter % 100000 ) == 0 ):
                    print( "- Hashed " + str( line_counter ) + " lines at " + str( datetime.datetime.now() ) )
                #-- END check to see if we've done 1000 records. --#

            #-- END loop over input lines.

        #-- END with ... to_hash_csv_file --#

    #-- END with ... hash_output_file --#    

#-- END loop over file list --#

print( "All files processed at " + str( datetime.datetime.now() ) )

# Evaluate

- Back to [Table of Contents](#Table-of-Contents)

Once the cell above has completed, you should now in your output folder (by default, the same directory as this notebook) have an output file named `hashed-<input_file_name>` per input file.